# Creation of the synthetic dataset

Due to troubles with the dataset of original/highlighted simplified dataset, the decision was made to create a synthetic dataset to train the adapters.

This notebook creates highlighted simplified texts from given original texts. You will get jsonl files that can directly be used to train and test the simplifier and highlighter of the Pipeline approach.
The files for the end2end approach can be created with the notebook create_synthetic_dataset_end2end.ipynb.

(You may find more details in the paper.)

In [2]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from tqdm import tqdm


MODEL_NAME = "google/gemma-2-9b-it"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"



###############################################################################
# LOAD MODEL
###############################################################################

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if DEVICE == "cuda" else torch.float32,
    device_map="auto"
)



ModuleNotFoundError: No module named 'transformers'

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# -------------------------------
# Model Configuration
# -------------------------------
MODEL_NAME = "LeoLM/leo-mistral-hessianai-7b-chat"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# -------------------------------
# Load Tokenizer
# -------------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# -------------------------------
# Load Model
# -------------------------------
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",  # Automatically place layers on available devices
    torch_dtype=torch.bfloat16 if DEVICE == "cuda" else torch.float32,
)

/home/cracle/data/conda/envs/training_gemma/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:18<00:00,  9.21s/it]


In [ ]:
# model test
inputs = tokenizer("Wie geht es dir?", return_tensors="pt").to(DEVICE)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=2048,
        temperature=0.1,
        do_sample=False
    )

raw = tokenizer.decode(output[0], skip_special_tokens=True)
print(raw)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Wie geht es dir? 
 assistant
Als KI habe ich keine Gefühle, daher ist es für mich nicht möglich, Emotionen zu erleben oder zu zeigen. Es geht mir einfach gut, danke für die Nachfrage. Wie geht es dir? 



## Prompts

To create the synthetic dataset, the pipeline approach of first simplyfing (using Prompt1 and instruction_simplifier) and then highlighting (using Prompt2 and instruction_highlighter) was used.

In [ ]:
PROMPT1= """<|im_start|>system
Du bist ein hilfreicher Assistent, der Texte bereinigt und vereinfacht, wobei wichtige Wörter hervorgehoben werden.
<|im_end|>
<|im_start|>user
Du erhältst einen Text.
Deine Aufgabe: Erstelle ein Textpaar mit einer sauberen Version des Originaltextes und einer vereinfachten Version dieses Textes.

Regeln:
- Das Textpaar besteht aus (orig, simple)
- Zuerst bereinige den Originaltext: Entferne alle Links, merkwürdige Formatierungen oder andere Artefakte aus der Vorverarbeitung. Es soll nur noch sauberer Text übrig bleiben, aber nichts umgeschrieben werden.
- Nun erstelle aus diesem sauberen Text einen entsprechenden vereinfachten Text.
- Der Text soll für Menschen mit kognitiven Beeinträchtigungen vereinfacht sein: Verwende sehr einfaches Vokabular, einfache Grammatik und wirklich sehr kurze Sätze.
- Jeder Satz sollte in einer neuen Zeile stehen.


WICHTIG:
- Gib nur ein JSON-Array aus.
- Format für das Textpaar genau wie folgt:

[
  {{"orig": sauberer Originaltext, "simple": vereinfachter Text}},
  ...
]

- Gib sonst nichts aus. Keine Erklärungen, keine Kommentare, kein zusätzlicher Text.
- Verwende doppelte Anführungszeichen für alle Schlüssel und String-Werte.
- Trenne mehrere Paare durch Kommata.

Originaler Text:
{orig}

Textpaar:
<|im_end|>
<|im_start|>assistant
"""


In [ ]:
instruction_simplifier = """
Du erhältst einen Text. Deine Aufgabe ist es, diesen Text zu bereinigen und zu vereinfachen.
- Zuerst bereinige den Originaltext: Entferne alle Links, merkwürdige Formatierungen oder andere Artefakte aus der Vorverarbeitung. Es soll nur noch sauberer Text übrig bleiben, aber nichts umgeschrieben werden.
- Nun erstelle aus diesem sauberen Text einen entsprechenden vereinfachten Text.
- Der Text soll für Menschen mit kognitiven Beeinträchtigungen vereinfacht sein: Verwende sehr einfaches Vokabular, einfache Grammatik und wirklich sehr kurze Sätze.
- Jeder Satz sollte in einer neuen Zeile stehen.
- Gib sonst nichts aus. Keine Erklärungen, keine Kommentare, kein zusätzlicher Text.
"""

In [ ]:
PROMPT2= """<|im_start|>system
Du bist ein hilfreicher Assistent, der Text in Markdown annotiert.
<|im_end|>
<|im_start|>user
Du erhältst einen Text.
Deine Aufgabe: Füge Markdown-Annotationen hinzu, ohne den Text inhaltlich zu verändern.  

Anweisungen:
- Annotiere Überschriften im Text mit Markdown (#, ## oder ###).
- Markiere alle wichtigen Wörter oder Phrasen fett (**so**).
- Halte dich strikt an das Markdown-Format.
- WICHTIG: Ändere oder schreibe den Text nicht um! 
- Füge nur die Markdown-Annotationen hinzu.
- Dein Output soll ausschließlich der originale Text mit den Markdown-Annotationen sein.
- Gib keine Erklärungen, Kommentare oder zusätzlichen Text aus.

Originaler Text:
{simple}

Textpaar:
<|im_end|>
<|im_start|>assistant
"""


In [ ]:
instruction_highlighter = """
Du erhältst einen Text. Deine Aufgabe ist es, Markdown-Annotationen hinzuzufügen, ohne den Text inhaltlich zu verändern.  
- Annotiere Überschriften im Text mit Markdown (#, ## oder ###).
- Markiere alle wichtigen Wörter oder Phrasen fett (**so**).
- Halte dich strikt an das Markdown-Format.
- WICHTIG: Ändere oder schreibe den Text nicht um! 
- Füge nur die Markdown-Annotationen hinzu.
- Dein Output soll ausschließlich der originale Text mit den Markdown-Annotationen sein.
- Gib keine Erklärungen, Kommentare oder zusätzlichen Text aus.
"""

In [ ]:
import json
from tqdm import tqdm
import traceback
import re
import json

def create_pair(orig_text):

    try:
        prompt = PROMPT1.replace("{orig}", orig_text)

        inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)

        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=4096,
                temperature=0.9,
                do_sample=True
            )

        raw = tokenizer.decode(output[0], skip_special_tokens=True)

        #print("raw model output:", raw)
        parsed = extract_json_from_text(raw)
        #print("json model output:", parsed)
        print("parsed", type(parsed))

        return parsed#json.dumps(parsed)
    
    except Exception as e:
        print("Error in pair:", e)
        traceback.print_exc()
        return None   # Ensure failure never crashes main loop

In [ ]:
def merge_json_list(json_list):
    """
    Merge a list of dicts like:
    [{'orig': '...', 'simple': '...'}, ...]
    into a single dict:
    {"orig": combined orig texts, "simple": combined simple texts}
    """
    if not json_list:
        return {"orig": "", "simple": ""}

    entry_dict = {}
    orig = ""
    simple = ""
    for d in json_list:
        orig_text = d["orig"]
        simple_text = d["simple"]
        orig = orig + "\\n"+ orig_text
        simple = simple + "\\n" + simple_text
    entry_dict["orig"] = orig
    entry_dict["simple"] = simple

    return entry_dict

In [ ]:
import ast
def extract_json_from_text(text):
    """
    Extracts the first JSON array from a text.
    Handles:
    - Whitespace/newlines
    - Trailing commas
    Returns a Python object (list/dict) if JSON is found, else None.
    """
    if not text or not text.strip():
        return None

    # Match everything between outer brackets
    pattern = re.compile(r'assistant.*\[\s*(\{.*\})\s*\]', re.DOTALL)
    match = pattern.search(text)
    if not match:
        return None
    
    output = match.group(0)

    #print("FOUNDJSON")
    #print(json_str)
        # Find the first '[' that starts the JSON array
    start_idx = output.find('[')
    if start_idx == -1:
        return None  # No JSON array found

    # Extract from the first '[' to the last ']' in the text
    end_idx = output.rfind(']')
    if end_idx == -1:
        return None  # Malformed JSON

    json_str = output[start_idx:end_idx+1]
    json_str = json_str.replace("\n", "\\n")
        # Detect if the string starts with '[\n' literally and remove the '\n'
    # Only remove backslash-n that is **right after [ or { or ,**
    json_str = re.sub(r'([\[\{\,])\\n', r'\1', json_str)
    json_str = re.sub(r'([\]\}\,])\\n', r'\1', json_str)
    json_str = json_str.replace("\"\\n", "\"")
    json_str = json_str.replace("{{", "{")
    json_str = json_str.replace("}}", "}")




    print("json_str", type(json_str))
    #print(json_str)
    try:
        # Convert string to Python object safely
        data = ast.literal_eval(json_str)
        if isinstance(data, list) and all(isinstance(item, dict) for item in data):
            fixed_json = data
        else:
            raise ValueError("The input string does not represent a list of dictionaries.")
    except Exception as e:
        raise ValueError(f"Could not parse the string: {e}")
        
    parsed = merge_json_list(fixed_json)

    print("merged", type(parsed))
    

    try:
        return parsed
    except json.JSONDecodeError as e:
        print(f"Failed to parse JSON: {e}")
        return None


In [ ]:
def highlight(text):
    try:
        prompt = PROMPT2.replace("{simple}", text)

        inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)

        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=4096,
                temperature=0.9,
                do_sample=True
            )

        raw = tokenizer.decode(output[0], skip_special_tokens=True)

        #print("raw model output:", type(raw))
        return raw
    
    except Exception as e:
        print("Error in highlight:", e)
        traceback.print_exc()
        return None   # Ensure failure never crashes main loop    

In [ ]:
def simplify(INPUT_FILE, SIMPLE_OUTPUT_FILE):
    torch.cuda.empty_cache()

    with open(INPUT_FILE, "r", encoding="utf8") as f:
        lines = [json.loads(l) for l in f]

    out = open(SIMPLE_OUTPUT_FILE, "w", encoding="utf8")

    failed_org_simple = []

    for entry in tqdm(lines):
        orig = entry["input"]

        pair = create_pair(orig)

        if pair == None:
            failed_org_simple.append(entry["id"])
            print("failed:", entry["id"])
            continue

        new_entry = {
                "id": entry.get("id"),
                "instruction": instruction_simplifier,#entry.get("instruction", ""),
                "input": pair["orig"].strip(),
                "output": pair["simple"].strip(),
                "metadata": entry.get("metadata", {})
            }
        out.write(json.dumps(new_entry, ensure_ascii=False) + "\n")

    out.close()

    print("These org_simple lines failed:")
    print(failed_org_simple)
    print("Done! Saved:", SIMPLE_OUTPUT_FILE)


In [ ]:
def highlight_entries(SIMPLE_OUTPUT_FILE, MARKED_OUTPUT_FILE):
    torch.cuda.empty_cache()

    with open(SIMPLE_OUTPUT_FILE, "r", encoding="utf8") as f2:
        lines2 = [json.loads(l) for l in f2]

    print(lines2[0])
    out2 = open(MARKED_OUTPUT_FILE, "w", encoding="utf8")

    failed_highlight = []
    
    for entry2 in tqdm(lines2):
        simple = entry2["output"]
        highlighted = highlight(simple)
        start_idx = highlighted.find('assistant')
        if start_idx == -1:
            highlighted_simple = None  # or handle error
        else:
            # Take everything from 'assistant' until the end
            highlighted_simple = highlighted[start_idx + len('assistant'):].lstrip()
            highlighted_simple = highlighted_simple.replace("```markdown", "") #
            highlighted_simple = highlighted_simple.replace("Originaler Text:", "") 
            highlighted_simple = highlighted_simple.replace("```", "") 



        if highlighted_simple == None:
            failed_highlight.append(entry2["id"])
            print("failed:", entry2["id"])
            continue
        new_entry = {
            "id": entry2.get("id"),
            "instruction": instruction_highlighter,#entry2.get("instruction", ""),
            "input": entry2["output"].strip(),
            "output": highlighted_simple,
            "metadata": entry2.get("metadata", {})
        }
        out2.write(json.dumps(new_entry, ensure_ascii=False) + "\n")

    out2.close()

If you the model skipped some lines in the first your run, you can can create files for a redo of just those failed ones with the following codeblock and rerun the model in the codeblock after this.

In [ ]:
# create new files with the entries that were missed in the last round
INPUT_FILE = "data/Accessibility_Seminar/datasets_for_models/simplifier_test.jsonl"         # Your original dataset

def load_ids(path):
    ids = set()
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            try:
                obj = json.loads(line)
                if "id" in obj:
                    ids.add(str(obj["id"]))
            except json.JSONDecodeError:
                print(f"Warning: Could not parse line in {path}")
    return ids

    
ids_a = load_ids("data/Accessibility_Seminar/datasets_for_models/simplifier_test.jsonl") # the original texts
ids_b = load_ids("data/Accessibility_Seminar/datasets_for_models_synthetic/simplifier_test.jsonl") # the entries in the synthetic set

only_in_a = sorted(ids_a - ids_b)
print("entries in a", len(ids_a))
print("entries skipped in b", len(only_in_a))

INPUT_FILE_part2 = "data/Accessibility_Seminar/datasets_for_models/simplifier_test_part2.jsonl"         # Your original dataset
SIMPLE_OUTPUT_FILE_part2 = "data/Accessibility_Seminar/datasets_for_models_synthetic/simplifier_test_part2.jsonl"   
MARKED_OUTPUT_FILE_part2 = "data/Accessibility_Seminar/datasets_for_models_synthetic/highlighter_test_part2.jsonl" 

# Write matching full records from A into output file
total_lines = 0
with open(INPUT_FILE, "r", encoding="utf-8") as f_in, \
    open(INPUT_FILE_part2, "w", encoding="utf-8") as f_out:

    for line in f_in:
        try:
            obj = json.loads(line)
        except json.JSONDecodeError:
            continue

        if str(obj.get("id")) in only_in_a:
            total_lines+=1
            f_out.write(json.dumps(obj, ensure_ascii=False) +"\n")
print(f"Total lines in part 2: {total_lines}")

entries in a 39
entries skipped in b 17
Total lines in part 2: 17


in val:


These org_simple lines failed:
['5456', '3536', '3429', '3245', '3963', '4018', '3230', '3309', '3458', '3420', '5446', '37', '3249', '3973', '3538', '5780', '5782', '3438']

In [ ]:
###############################################################################
# MAIN PROCESSING LOOP
###############################################################################

from tqdm import tqdm
import json

INPUT_FILE = "data/Accessibility_Seminar/datasets_for_models/simplifier_test_part2.jsonl"         # Your original dataset
SIMPLE_OUTPUT_FILE = "data/Accessibility_Seminar/datasets_for_models_synthetic/simplifier_test_part2.jsonl"   # Cleaned and aligned pairs
MARKED_OUTPUT_FILE = "data/Accessibility_Seminar/datasets_for_models_synthetic/highlighter_test_part2.jsonl" 


def main():

    simplify(INPUT_FILE, SIMPLE_OUTPUT_FILE)
    highlight_entries(SIMPLE_OUTPUT_FILE, MARKED_OUTPUT_FILE)

    print("DONE")

if __name__ == "__main__":
    main()


  6%|▌         | 1/17 [00:07<02:02,  7.63s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


parsed <class 'NoneType'>
failed: 5788


 12%|█▏        | 2/17 [00:12<01:31,  6.07s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


parsed <class 'NoneType'>
failed: 3530


 18%|█▊        | 3/17 [00:19<01:28,  6.33s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

INPUT
\nAllgemeine Verpflichtungen
In Artikel 4 der Behindertenrechtskonvention sind die "Allgemeinen Verpflichtungen" des Staates aufgeführt. Es sind dort verschiedene Punkte benannt, die der Vertragsstaat erfüllen muss. Eine grundlegende Verpflichtung besteht darin, "Disability Mainstreaming" einzuführen, was bedeutet, dass der Schutz und die Förderung der Rechte von Menschen mit Behinderungen bei allen politischen Konzepten und Programmen zu berücksichtigen ist. Die Staaten haben aber auch die Verpflichtung, Gesetze und Verordnungen anzupassen und das 
konventionsgemäße Handeln
 durch alle öffentlichen Aufgabenträger (Behörden, Sozialversicherungen) sicherzustellen. Im Bereich der 
Forschung und Entwicklung
 für Güter, Dienstleistungen, Geräte und Einrichtungen in universellem Design und als unterstützende Technologien hat der Staat dafür zu sorgen, dass diese Forschung stattfindet und dass die Informationen über solche Produkte auch den Menschen zur Verfügung stehen. Ebenfalls hat 

 80%|████████  | 4/5 [00:22<00:06,  6.56s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INPUT
\ntaz leicht sagt erstmal "Tschüss"
Es war ein spannendes Jahr für taz leicht.
Im Sommer 2017 begannen wir mit etwas so noch nicht Dagewesenem in den deutschen Medien: Wir übersetzten zur Bundestagswahl meinungsstarke taz-Texte in Leichte Sprache, um auch jene LeserInnen zu erreichen, für die das "Standardprogramm" unserer Zeitung zu schwer zu lesen ist.
Seither haben wir über 70 taz-Texte in kurze Sätze übersetzt und damit Leichter Sprache und ihren LeserInnen zu mehr Sichtbarkeit verholfen.
Sie brauchen taz leicht nicht allein zu lesen! Wir haben viele Rückmeldungen von ganz unterschiedlichen LeserInnen bekommen – von Menschen mit Lernschwierigkeiten, Angehörigen, persönlichen Assistenzen, Förderschulen, Bibliotheken, SprachlehrerInnen, anderen Leichte-Sprache-Projekten, HelferInnen in der Flüchtlingshilfe, Menschen, die gerade Deutsch lernen und Eltern, die die Texte ihren Kindern zu lesen geben.
Dieselbe Zeitung lesen, die auch alle anderen lesen
Eine Frau schrieb uns – ihr M

100%|██████████| 5/5 [00:32<00:00,  6.46s/it]

INPUT
\nWir stellen Ihnen unsere Partner vor.\nDas ist eine Liste unserer Partner. Wir arbeiten mit ihnen zusammen, um Ihnen eine bessere Erfahrung zu bieten.\nWir stellen Ihnen 1a Zugang vor.\nWir bieten Beratung für Unternehmen und Institutionen an.\n1a Zugang hilft Menschen mit Behinderungen, damit sie einen einfachen Zugang zu Dingen haben.\n1a Zugang arbeitet mit Unternehmen zusammen, um Dinge leicht zugänglich zu machen.\nSie können mehr über 1a Zugang auf ihrer Website erfahren.\nWir stellen Ihnen capito Stuttgart vor.\ncapito Stuttgart hilft Menschen, Texte leicht zu lesen und zu verstehen.\ncapito macht Texte einfach zu lesen.\ncapito arbeitet mit Übersetzungen, um Texte einfach zu machen.\nSie können capito Stuttgart unter dieser Adresse finden.\nWir stellen Ihnen Femos gGmbH vor.\nFemos ist eine gemeinnützige Organisation. Sie hilft Menschen und macht dabei keine Gewinne.\nFemos wird von Wilhelm Kohlberger geleitet.\nSie können Femos unter dieser Adresse finden.\nWir stellen